In [4]:
import pandas as pd
import os
import numpy as np

In [5]:
path = "C:/Users/doosti/Dropbox (Chapman)/Research/Research Projects/Fitness/Data/"
file = "labeled_comments.csv"

In [20]:
comments_labeled = pd.read_csv(os.path.join(path, file))
comments = pd.read_csv(os.path.join(path, "merged_comments.csv"))
videos = pd.read_csv(os.path.join(path, "videos_101223.csv"))
# matching with the creator id
comments['creator_id'] = comments.merge(videos,how='left',on='video_id').creator_id.values
comments_labeled['creator_id'] = comments_labeled.merge(videos,how='left',on='video_id').creator_id.values

C:\Users\doosti\AppData\Local\Temp\ipykernel_195840\2430294057.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  comments_labeled = pd.read_csv(os.path.join(path, file))


In [43]:
print((comments.creator_id==comments_labeled.creator_id).sum(),comments.shape[0])
print(comments.isnull().sum())
print(comments_labeled.isnull().sum())

809335 827580
video_id             0
user_id              0
comment_text       100
no_likes            14
no_replies          14
comment_year        14
comment_month       28
comment_day         28
creator_id       18245
dtype: int64
video_id              0
user_id               0
comment_text        100
no_likes             14
no_replies           14
comment_year         14
comment_month        28
comment_day          28
label            764525
label_score      764525
creator_id        18245
dtype: int64


In [18]:
print(comments_labeled.columns)
print(comments.columns)

Index(['video_id', 'user_id', 'comment_text', 'no_likes', 'no_replies',
       'comment_year', 'comment_month', 'comment_day', 'label', 'label_score'],
      dtype='object')
Index(['video_id', 'user_id', 'comment_text', 'no_likes', 'no_replies',
       'comment_year', 'comment_month', 'comment_day', 'creator_id'],
      dtype='object')


In [44]:
# threshold = 11
comments_labeled = comments_labeled[comments_labeled.video_id.str.len() == 11].copy()
print(comments_labeled.shape)
# drop rows with missing year, month, or day
comments_labeled = comments_labeled.dropna(subset=['comment_year', 'comment_month', 'comment_day'])
print(comments_labeled.shape)
print(comments_labeled.isnull().sum())

(827566, 11)
(827552, 11)
video_id              0
user_id               0
comment_text        100
no_likes              0
no_replies            0
comment_year          0
comment_month         0
comment_day           0
label            764497
label_score      764497
creator_id        18231
dtype: int64


In [48]:
# Create a function to format and convert the date
def to_datetime_with_padding(year, month, day):
    # Pad month and day with zeros if necessary
    month = f"{month:02d}"
    day = f"{day:02d}"
    # Combine and convert to datetime
    date_str = f"{year}-{month}-{day}"
    return pd.to_datetime(date_str)

comments_labeled['comment_year'] = comments_labeled.comment_year.astype(int)
comments_labeled['comment_month'] = comments_labeled.comment_month.astype(int)
comments_labeled['comment_day'] = comments_labeled.comment_day.astype(int)

# Apply the function to each row
comments_labeled['date'] = comments_labeled.apply(lambda row: to_datetime_with_padding(row["comment_year"], row["comment_month"], row["comment_day"]), axis=1)


In [52]:

channel_day = comments_labeled.groupby(['creator_id','date']).label.value_counts().unstack().reset_index()
channel_day = channel_day.fillna(0)
channel_day.head()

label,creator_id,date,appearance,competence,fitness,interest,social
0,262h9yTG1f03Nobo501,2020-11-11,0.0,3.0,0.0,0.0,0.0
1,262h9yTG1f03Nobo501,2020-11-12,0.0,9.0,0.0,0.0,0.0
2,262h9yTG1f03Nobo501,2020-11-13,0.0,6.0,1.0,1.0,0.0
3,262h9yTG1f03Nobo501,2020-11-14,0.0,11.0,0.0,1.0,0.0
4,262h9yTG1f03Nobo501,2020-11-15,0.0,11.0,2.0,0.0,0.0


In [53]:
channel_day.to_csv(os.path.join(path, "channel_day.csv"), index=False)

## Creating a balanced data set

In [6]:
channel_day = pd.read_csv(os.path.join(path, "channel_day.csv"))
trends = pd.read_csv(os.path.join(path, "final_yoga_channels_trends_091523.csv"))

In [7]:
trends.head()

,creator_id,date,views,engagements,uploads,followers
0,YhItxj8Ap9,2020-09-15,8173,153,0,NaN
1,YhItxj8Ap9,2020-09-16,9924,392,2,NaN
2,YhItxj8Ap9,2020-09-17,9976,369,1,NaN
3,YhItxj8Ap9,2020-09-18,9279,223,0,NaN
4,YhItxj8Ap9,2020-09-19,8484,191,1,NaN


In [8]:
channel_day.head()

,creator_id,date,appearance,competence,fitness,interest,social
0,262h9yTG1f03Nobo501,2020-11-11,0.0,3.0,0.0,0.0,0.0
1,262h9yTG1f03Nobo501,2020-11-12,0.0,9.0,0.0,0.0,0.0
2,262h9yTG1f03Nobo501,2020-11-13,0.0,6.0,1.0,1.0,0.0
3,262h9yTG1f03Nobo501,2020-11-14,0.0,11.0,0.0,1.0,0.0
4,262h9yTG1f03Nobo501,2020-11-15,0.0,11.0,2.0,0.0,0.0


In [18]:
merged = trends.merge(channel_day,how='left',on=['creator_id','date']) #.to_csv(os.path.join(path, "trends_channel_day.csv"), index=False)
merged.isnull().sum()
print(f"number of creators with comments: {merged[merged.appearance.notnull()].creator_id.nunique()}, out of {merged.creator_id.nunique()} creators in trends")
# there seems to be a problem with the data set, the number of creators with comments is less than the number of creators in trends

number of creators with comments: 23, out of 75 creators in trends


In [26]:
# inspection
print(f"number of unique creators in the comments data: {comments_labeled.creator_id.nunique()}")
print(f"number of unique creators in channel-day data: {channel_day.creator_id.nunique()}")
# there must be some creators in the comments data that are not in the channel-day data, potentially a merging issue

number of unique creators in the comments data: 71
number of unique creators in channel-day data: 23


In [27]:
# check if the issue is with "date"
merged = trends.merge(channel_day,how='left',on=['creator_id'])
print(merged[merged.appearance.notnull()].creator_id.nunique())
# no, the issue is not with "date"

23


In [37]:
# there might be a mismatch between the creator_id in the trends and the comments data
creator_ids = pd.read_csv(os.path.join(path, "channel_ids_updated.csv"))
print(creator_ids.head())
print(creator_ids.shape)
channels = pd.read_csv(os.path.join(path, "final_channel_list.csv"))
print(channels.head())
print(channels.shape)

                       channel_name                channel_id  Unnamed: 2  \
0                        Chloe Ting  UCCgLoMYIyP0U56dEhEL1wXQ         NaN   
1                       Pamela Reif  UChVRfsT_ASBZk10o0An7Ucg         NaN   
2                       Lilly Sabri  UCTsM1dSAiXqiV5oZjuNw_Bg         NaN   
3  Walk at home with Leslie Sansone  UCVl6ZdslZz2Zj-34bMJFPbg         NaN   
4                        Workout 4D  UC8-9mdGBP5dtxEKJM2E9bHA         NaN   

  yt_channel_id channel_title  
0    8MVOSriENi    Chloe Ting  
1    H51NdBdKnh   Pamela Reif  
2    YuTB3vwpGD   Lilly Sabri  
3    2BkNULnxfr  Walk at Home  
4    cYe6OSjhG8    Workout 4D  
(89, 5)
                 channel_name          tubular_id
0            Jessica Richburg          OYTNzS4c3W
1              Yoga with bird          tsY9pcTzPa
2            Move with Nicole  2oCHNAExQ03RWMs201
3  Yoga with Allie Van Fossen          yKNyfr6lOY
4                Travis Eliot          A5YFIsqHCL
(76, 2)


In [36]:
channels.merge(creator_ids,how='inner',left_on='channel_name',right_on='channel_title').shape

(8, 7)